In [65]:
print("ok")

ok


In [66]:
from langchain import PromptTemplate

In [67]:
from langchain.chains import RetrievalQA

In [68]:
!pip install langchain-community


In [69]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import Pinecone as PineconeStor
#import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [70]:
from langchain.llms import ctransformers

### Extract my data from Pdf file

In [71]:
def load_pdf(data):
    loader=DirectoryLoader(data, 
                    glob="*.pdf",
                    loader_cls=PyMuPDFLoader)
    documents= loader.load()

    return documents

In [72]:
extracted_data = load_pdf("data/")

c:\Users\SS\anaconda3\envs\mchb\Lib\site-packages\langchain_community\document_loaders\parsers\pdf.py:299: UserWarning: Warning: Empty content on page 0 of document data\Medical_book.pdf
  warnings.warn(


### Split Text to Create chunks

In [73]:
def text_splinting(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks 

In [74]:
chunks = text_splinting(extracted_data)

In [75]:
len(chunks)

5779

### Creating Embedings from chunks

In [76]:

def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [77]:
embedding = download_hugging_face_embeddings()

In [78]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### Save Vetoers to pinecone database    

In [79]:
!pip install python-dotenv

In [91]:
import os 
from  dotenv import load_dotenv
#---------------------------------------------------------#
#---------- Load the environment variable ----------------#
#---------------------------------------------------------#
load_dotenv()
key = os.getenv("PINION_KEY")
environment = os.getenv("PINECONE_ENVIRONMENT")
print(key, environment)
#---------------------------------------------------------#
#---------------------------------------------------------#


os.environ['PINECONE_API_KEY'] = key
os.environ['PINECONE_API_ENV'] = environment

pc=pinecone.Pinecone(api_key =key, environment=environment)
index_name="medicalchatbot"

a141cb5f-80bb-473e-9658-55eb0dcf491b us-east-1


In [92]:
indexes=pc.list_indexes()
indexes

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 384,
              'host': 'medicalchatbot-d4dxd7p.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'medicalchatbot',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [93]:
if index_name in pc.list_indexes().names():
 
    docserach = PineconeStor.from_texts([t.page_content for t in chunks], embedding, index_name=index_name)
else :
    print("not in ")
    


### Query Some Data 

In [95]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embedding)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(id='e97ca4ae-0948-4299-a674-7c749c188333', metadata={}, page_content='to commonly encountered environmental substances.\nPurpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(id='64eeef6e-245e-4c7d-8e6e-15a8061748de', metadata={}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shor

In [96]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [97]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [101]:
!pip install ctransformers

   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.9 MB ? eta -:--:--
   -------- ------------------------------- 2.1/9.9 MB 8.4 MB/s eta 0:00:01
   --------------- ------------------------ 3.9/9.9 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.9 MB 14.7 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 14.6 MB/s eta 0:00:00


In [103]:
llm=ctransformers.CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [104]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])